In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


## Importing Packages

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from itertools import product
from datetime import timedelta

In [3]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cpu'

## Loading Dataset

In [4]:
df = pd.read_csv('../DataFrames/final_merged_copper_dataset.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df["Date"] = pd.to_datetime(df["Date"])
df

,Date,Price,Month,Year,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM,DEXCHUS
0,2000-01-04,0.8480,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120774
1,2000-01-05,0.8565,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120776
2,2000-01-06,0.8530,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120777
3,2000-01-07,0.8540,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120782
4,2000-01-10,0.8465,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120782
...,...,...,...,...,...,...,...,...,...
6047,2023-11-20,3.8335,11.0,2023.0,19374.000,307.0,6.600000,168.657489,0.139509
6048,2023-11-21,3.8395,11.0,2023.0,19374.000,307.0,6.600000,168.657489,0.140150
6049,2023-11-22,3.7970,11.0,2023.0,19374.000,307.0,6.600000,168.657489,0.139762
6050,2023-11-23,3.7743,11.0,2023.0,19374.000,307.0,6.600000,168.657489,0.139823


In [5]:
df.to_csv("df.csv",index=False)

In [6]:
last_date = df['Date'].max().date()
last_date

datetime.date(2023, 11, 24)

In [7]:
df

,Date,Price,Month,Year,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM,DEXCHUS
0,2000-01-04,0.8480,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120774
1,2000-01-05,0.8565,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120776
2,2000-01-06,0.8530,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120777
3,2000-01-07,0.8540,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120782
4,2000-01-10,0.8465,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120782
...,...,...,...,...,...,...,...,...,...
6047,2023-11-20,3.8335,11.0,2023.0,19374.000,307.0,6.600000,168.657489,0.139509
6048,2023-11-21,3.8395,11.0,2023.0,19374.000,307.0,6.600000,168.657489,0.140150
6049,2023-11-22,3.7970,11.0,2023.0,19374.000,307.0,6.600000,168.657489,0.139762
6050,2023-11-23,3.7743,11.0,2023.0,19374.000,307.0,6.600000,168.657489,0.139823


In [8]:
LOOKBACK = 15
data = pd.DataFrame({'Date': df["Date"].copy(),
                     'Price': df["Price"].copy()})
# Select the "Date" column and store it in the variable 'train_dates'.
dates = data["Date"]

# Exclude the first 'LOOKBACK' dates from 'train_dates' to create a time window.
dates = dates[LOOKBACK:]
dates

15     2000-01-26
16     2000-01-27
17     2000-01-28
18     2000-01-31
19     2000-02-01
          ...    
6047   2023-11-20
6048   2023-11-21
6049   2023-11-22
6050   2023-11-23
6051   2023-11-24
Name: Date, Length: 6037, dtype: datetime64[ns]

In [9]:
TRAIN_DATA_LEN = dates.shape[0]

In [10]:
dates.reset_index(drop=True, inplace=True)

In [11]:
dates

0      2000-01-26
1      2000-01-27
2      2000-01-28
3      2000-01-31
4      2000-02-01
          ...    
6032   2023-11-20
6033   2023-11-21
6034   2023-11-22
6035   2023-11-23
6036   2023-11-24
Name: Date, Length: 6037, dtype: datetime64[ns]

In [12]:
from copy import deepcopy as dc


def prepare_dataframe_for_lstm(df, n_steps):
    df = dc(df)

    #df.set_index('Date', inplace=True)

    for i in range(1, n_steps + 1):
        df[f'Price(t-{i})'] = df['Price'].shift(i)

    df.dropna(inplace=True)

    return df


shifted_df = prepare_dataframe_for_lstm(data, LOOKBACK)
shifted_df = shifted_df.reset_index(drop=True)
df = df.drop(df.index[-LOOKBACK:])
df

,Date,Price,Month,Year,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM,DEXCHUS
0,2000-01-04,0.8480,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120774
1,2000-01-05,0.8565,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120776
2,2000-01-06,0.8530,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120777
3,2000-01-07,0.8540,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120782
4,2000-01-10,0.8465,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120782
...,...,...,...,...,...,...,...,...,...
6032,2023-10-30,3.6585,10.0,2023.0,19374.000,307.0,6.600000,168.657489,0.136737
6033,2023-10-31,3.6490,10.0,2023.0,19374.000,307.0,6.600000,168.657489,0.136676
6034,2023-11-01,3.6490,11.0,2023.0,19374.000,307.0,6.600000,168.657489,0.136659
6035,2023-11-02,3.6725,11.0,2023.0,19374.000,307.0,6.600000,168.657489,0.136687


In [13]:
shifted_df

,Date,Price,Price(t-1),Price(t-2),Price(t-3),Price(t-4),Price(t-5),Price(t-6),Price(t-7),Price(t-8),Price(t-9),Price(t-10),Price(t-11),Price(t-12),Price(t-13),Price(t-14),Price(t-15)
0,2000-01-26,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530,0.8565,0.8480
1,2000-01-27,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530,0.8565
2,2000-01-28,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530
3,2000-01-31,0.8545,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540
4,2000-02-01,0.8480,0.8545,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6032,2023-11-20,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,3.6380,3.6790,3.7190,3.6815,3.6725,3.6490,3.6490,3.6585
6033,2023-11-21,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,3.6380,3.6790,3.7190,3.6815,3.6725,3.6490,3.6490
6034,2023-11-22,3.7970,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,3.6380,3.6790,3.7190,3.6815,3.6725,3.6490
6035,2023-11-23,3.7743,3.7970,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,3.6380,3.6790,3.7190,3.6815,3.6725


In [14]:
shifted_df["Real_GDP"] = df["Real_GDP"]
shifted_df["CPI"] = df["CPI"]
shifted_df["inflation_rate"] = df["inflation_rate"]
shifted_df["PALLFNFINDEXM"] = df["PALLFNFINDEXM"]
shifted_df["DEXCHUS"] = df["DEXCHUS"]
shifted_df

,Date,Price,Price(t-1),Price(t-2),Price(t-3),Price(t-4),Price(t-5),Price(t-6),Price(t-7),Price(t-8),...,Price(t-11),Price(t-12),Price(t-13),Price(t-14),Price(t-15),Real_GDP,CPI,inflation_rate,PALLFNFINDEXM,DEXCHUS
0,2000-01-26,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,0.8565,...,0.8465,0.8540,0.8530,0.8565,0.8480,14096.033,168.8,3.376857,59.470000,0.120774
1,2000-01-27,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,...,0.8435,0.8465,0.8540,0.8530,0.8565,14096.033,168.8,3.376857,59.470000,0.120776
2,2000-01-28,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,...,0.8660,0.8435,0.8465,0.8540,0.8530,14096.033,168.8,3.376857,59.470000,0.120777
3,2000-01-31,0.8545,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,...,0.8565,0.8660,0.8435,0.8465,0.8540,14096.033,168.8,3.376857,59.470000,0.120782
4,2000-02-01,0.8480,0.8545,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,...,0.8625,0.8565,0.8660,0.8435,0.8465,14096.033,168.8,3.376857,59.470000,0.120782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6032,2023-11-20,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,3.6380,...,3.6815,3.6725,3.6490,3.6490,3.6585,19374.000,307.0,6.600000,168.657489,0.136737
6033,2023-11-21,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,...,3.7190,3.6815,3.6725,3.6490,3.6490,19374.000,307.0,6.600000,168.657489,0.136676
6034,2023-11-22,3.7970,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,...,3.6790,3.7190,3.6815,3.6725,3.6490,19374.000,307.0,6.600000,168.657489,0.136659
6035,2023-11-23,3.7743,3.7970,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,...,3.6380,3.6790,3.7190,3.6815,3.6725,19374.000,307.0,6.600000,168.657489,0.136687


In [15]:
price_columns = [col for col in shifted_df.columns if 'Price' in col]
price_columns

['Price',
 'Price(t-1)',
 'Price(t-2)',
 'Price(t-3)',
 'Price(t-4)',
 'Price(t-5)',
 'Price(t-6)',
 'Price(t-7)',
 'Price(t-8)',
 'Price(t-9)',
 'Price(t-10)',
 'Price(t-11)',
 'Price(t-12)',
 'Price(t-13)',
 'Price(t-14)',
 'Price(t-15)']

In [16]:
prices_before_scaling_as_np = shifted_df[price_columns].values
prices_before_scaling_as_np

array([[0.8415, 0.8425, 0.8565, ..., 0.853 , 0.8565, 0.848 ],
       [0.8445, 0.8415, 0.8425, ..., 0.854 , 0.853 , 0.8565],
       [0.84  , 0.8445, 0.8415, ..., 0.8465, 0.854 , 0.853 ],
       ...,
       [3.797 , 3.8395, 3.8335, ..., 3.6815, 3.6725, 3.649 ],
       [3.7743, 3.797 , 3.8395, ..., 3.719 , 3.6815, 3.6725],
       [3.8205, 3.7743, 3.797 , ..., 3.679 , 3.719 , 3.6815]])

In [17]:
prices_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_prices = prices_scaler.fit_transform(prices_before_scaling_as_np)
scaled_prices

array([[-0.89235389, -0.89189189, -0.88542389, ..., -0.88704089,
        -0.88542389, -0.88935089],
       [-0.89096789, -0.89235389, -0.89189189, ..., -0.88657889,
        -0.88704089, -0.88542389],
       [-0.89304689, -0.89096789, -0.89235389, ..., -0.89004389,
        -0.88657889, -0.88704089],
       ...,
       [ 0.47308847,  0.49272349,  0.48995149, ...,  0.41972742,
         0.41556942,  0.4047124 ],
       [ 0.46260106,  0.47308847,  0.49272349, ...,  0.43705244,
         0.41972742,  0.41556942],
       [ 0.48394548,  0.46260106,  0.47308847, ...,  0.41857242,
         0.43705244,  0.41972742]])

In [97]:
scaled_prices_flipped = np.fliplr(scaled_prices)
scaled_prices_flipped

array([[-0.88935089, -0.88542389, -0.88704089, ..., -0.88542389,
        -0.89189189, -0.89235389],
       [-0.88542389, -0.88704089, -0.88657889, ..., -0.89189189,
        -0.89235389, -0.89096789],
       [-0.88704089, -0.88657889, -0.89004389, ..., -0.89235389,
        -0.89096789, -0.89304689],
       ...,
       [ 0.4047124 ,  0.41556942,  0.41972742, ...,  0.48995149,
         0.49272349,  0.47308847],
       [ 0.41556942,  0.41972742,  0.43705244, ...,  0.49272349,
         0.47308847,  0.46260106],
       [ 0.41972742,  0.43705244,  0.41857242, ...,  0.47308847,
         0.46260106,  0.48394548]])

In [18]:
scaled_prices.shape

(6037, 16)

In [19]:
GDP_before_scaling_as_np = shifted_df["Real_GDP"].to_numpy()
CPI_before_scaling_as_np = shifted_df["CPI"].to_numpy()
inflation_rate_before_scaling_as_np = shifted_df["inflation_rate"].to_numpy()
PALLFNFINDEXM_before_scaling_as_np = shifted_df["PALLFNFINDEXM"].to_numpy()
DEXCHUS = shifted_df["DEXCHUS"].to_numpy()

In [20]:
GDP_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_GDP = GDP_scaler.fit_transform(GDP_before_scaling_as_np.reshape(-1, 1))
scaled_GDP

array([[-1.        ],
       [-1.        ],
       [-1.        ],
       ...,
       [ 0.36628638],
       [ 0.36628638],
       [ 0.36628638]])

In [21]:
CPI_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_CPI = CPI_scaler.fit_transform(CPI_before_scaling_as_np.reshape(-1, 1))
scaled_CPI

array([[-1.        ],
       [-1.        ],
       [-1.        ],
       ...,
       [ 0.98864658],
       [ 0.98864658],
       [ 0.98864658]])

In [22]:
IR_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_IR = IR_scaler.fit_transform(inflation_rate_before_scaling_as_np.reshape(-1, 1))
scaled_IR

array([[-0.10690381],
       [-0.10690381],
       [-0.10690381],
       ...,
       [ 0.66433555],
       [ 0.66433555],
       [ 0.66433555]])

In [23]:
PALLFNFINDEXM_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_PALLFNFINDEXM = PALLFNFINDEXM_scaler.fit_transform(PALLFNFINDEXM_before_scaling_as_np.reshape(-1, 1))
scaled_PALLFNFINDEXM

array([[-0.88354852],
       [-0.88354852],
       [-0.88354852],
       ...,
       [ 0.24367179],
       [ 0.24367179],
       [ 0.24367179]])

In [24]:
DEXCHUS_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_DEXCHUS = DEXCHUS_scaler.fit_transform(DEXCHUS.reshape(-1, 1))
scaled_DEXCHUS

array([[-0.99993486],
       [-0.99986972],
       [-0.99980458],
       ...,
       [-0.29056913],
       [-0.28931784],
       [-0.27635776]])

In [25]:

stacked_np = np.hstack((scaled_prices, scaled_GDP,scaled_CPI,scaled_IR,scaled_PALLFNFINDEXM,scaled_DEXCHUS))

# Display the combined NumPy array
stacked_np.shape


(6037, 21)

In [26]:
scaled_prices.shape

(6037, 16)

In [27]:
scaled_df = shifted_df.drop(['Date'], axis=1)
scaled_df

,Price,Price(t-1),Price(t-2),Price(t-3),Price(t-4),Price(t-5),Price(t-6),Price(t-7),Price(t-8),Price(t-9),...,Price(t-11),Price(t-12),Price(t-13),Price(t-14),Price(t-15),Real_GDP,CPI,inflation_rate,PALLFNFINDEXM,DEXCHUS
0,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,0.8565,0.8660,...,0.8465,0.8540,0.8530,0.8565,0.8480,14096.033,168.8,3.376857,59.470000,0.120774
1,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,0.8565,...,0.8435,0.8465,0.8540,0.8530,0.8565,14096.033,168.8,3.376857,59.470000,0.120776
2,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,...,0.8660,0.8435,0.8465,0.8540,0.8530,14096.033,168.8,3.376857,59.470000,0.120777
3,0.8545,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,...,0.8565,0.8660,0.8435,0.8465,0.8540,14096.033,168.8,3.376857,59.470000,0.120782
4,0.8480,0.8545,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,...,0.8625,0.8565,0.8660,0.8435,0.8465,14096.033,168.8,3.376857,59.470000,0.120782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6032,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,3.6380,3.6790,...,3.6815,3.6725,3.6490,3.6490,3.6585,19374.000,307.0,6.600000,168.657489,0.136737
6033,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,3.6380,...,3.7190,3.6815,3.6725,3.6490,3.6490,19374.000,307.0,6.600000,168.657489,0.136676
6034,3.7970,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,...,3.6790,3.7190,3.6815,3.6725,3.6490,19374.000,307.0,6.600000,168.657489,0.136659
6035,3.7743,3.7970,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,...,3.6380,3.6790,3.7190,3.6815,3.6725,19374.000,307.0,6.600000,168.657489,0.136687


In [28]:
flipped_df = shifted_df[shifted_df.columns[::-1]]
flipped_df = flipped_df.drop(['Date'], axis=1)
flipped_df

,DEXCHUS,PALLFNFINDEXM,inflation_rate,CPI,Real_GDP,Price(t-15),Price(t-14),Price(t-13),Price(t-12),Price(t-11),...,Price(t-9),Price(t-8),Price(t-7),Price(t-6),Price(t-5),Price(t-4),Price(t-3),Price(t-2),Price(t-1),Price
0,0.120774,59.470000,3.376857,168.8,14096.033,0.8480,0.8565,0.8530,0.8540,0.8465,...,0.8660,0.8565,0.8625,0.8660,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415
1,0.120776,59.470000,3.376857,168.8,14096.033,0.8565,0.8530,0.8540,0.8465,0.8435,...,0.8565,0.8625,0.8660,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415,0.8445
2,0.120777,59.470000,3.376857,168.8,14096.033,0.8530,0.8540,0.8465,0.8435,0.8660,...,0.8625,0.8660,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415,0.8445,0.8400
3,0.120782,59.470000,3.376857,168.8,14096.033,0.8540,0.8465,0.8435,0.8660,0.8565,...,0.8660,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415,0.8445,0.8400,0.8545
4,0.120782,59.470000,3.376857,168.8,14096.033,0.8465,0.8435,0.8660,0.8565,0.8625,...,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415,0.8445,0.8400,0.8545,0.8480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6032,0.136737,168.657489,6.600000,307.0,19374.000,3.6585,3.6490,3.6490,3.6725,3.6815,...,3.6790,3.6380,3.6405,3.5870,3.6660,3.6835,3.7185,3.7025,3.7345,3.8335
6033,0.136676,168.657489,6.600000,307.0,19374.000,3.6490,3.6490,3.6725,3.6815,3.7190,...,3.6380,3.6405,3.5870,3.6660,3.6835,3.7185,3.7025,3.7345,3.8335,3.8395
6034,0.136659,168.657489,6.600000,307.0,19374.000,3.6490,3.6725,3.6815,3.7190,3.6790,...,3.6405,3.5870,3.6660,3.6835,3.7185,3.7025,3.7345,3.8335,3.8395,3.7970
6035,0.136687,168.657489,6.600000,307.0,19374.000,3.6725,3.6815,3.7190,3.6790,3.6380,...,3.5870,3.6660,3.6835,3.7185,3.7025,3.7345,3.8335,3.8395,3.7970,3.7743


In [29]:
flipped_df.reset_index(drop=True, inplace=True)

In [30]:
df.drop(columns=['Price'], inplace=True)

In [31]:
new_df = pd.concat([df, flipped_df], axis=1)

In [32]:
new_df

,Date,Month,Year,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM,DEXCHUS,DEXCHUS,PALLFNFINDEXM,...,Price(t-9),Price(t-8),Price(t-7),Price(t-6),Price(t-5),Price(t-4),Price(t-3),Price(t-2),Price(t-1),Price
0,2000-01-04,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120774,0.120774,59.470000,...,0.8660,0.8565,0.8625,0.8660,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415
1,2000-01-05,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120776,0.120776,59.470000,...,0.8565,0.8625,0.8660,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415,0.8445
2,2000-01-06,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120777,0.120777,59.470000,...,0.8625,0.8660,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415,0.8445,0.8400
3,2000-01-07,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120782,0.120782,59.470000,...,0.8660,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415,0.8445,0.8400,0.8545
4,2000-01-10,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120782,0.120782,59.470000,...,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415,0.8445,0.8400,0.8545,0.8480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6032,2023-10-30,10.0,2023.0,19374.000,307.0,6.600000,168.657489,0.136737,0.136737,168.657489,...,3.6790,3.6380,3.6405,3.5870,3.6660,3.6835,3.7185,3.7025,3.7345,3.8335
6033,2023-10-31,10.0,2023.0,19374.000,307.0,6.600000,168.657489,0.136676,0.136676,168.657489,...,3.6380,3.6405,3.5870,3.6660,3.6835,3.7185,3.7025,3.7345,3.8335,3.8395
6034,2023-11-01,11.0,2023.0,19374.000,307.0,6.600000,168.657489,0.136659,0.136659,168.657489,...,3.6405,3.5870,3.6660,3.6835,3.7185,3.7025,3.7345,3.8335,3.8395,3.7970
6035,2023-11-02,11.0,2023.0,19374.000,307.0,6.600000,168.657489,0.136687,0.136687,168.657489,...,3.5870,3.6660,3.6835,3.7185,3.7025,3.7345,3.8335,3.8395,3.7970,3.7743


In [33]:
new_df.drop(columns=['Month', 'Year'], inplace=True)
new_df

,Date,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM,DEXCHUS,DEXCHUS,PALLFNFINDEXM,inflation_rate,CPI,...,Price(t-9),Price(t-8),Price(t-7),Price(t-6),Price(t-5),Price(t-4),Price(t-3),Price(t-2),Price(t-1),Price
0,2000-01-04,14096.033,168.8,3.376857,59.470000,0.120774,0.120774,59.470000,3.376857,168.8,...,0.8660,0.8565,0.8625,0.8660,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415
1,2000-01-05,14096.033,168.8,3.376857,59.470000,0.120776,0.120776,59.470000,3.376857,168.8,...,0.8565,0.8625,0.8660,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415,0.8445
2,2000-01-06,14096.033,168.8,3.376857,59.470000,0.120777,0.120777,59.470000,3.376857,168.8,...,0.8625,0.8660,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415,0.8445,0.8400
3,2000-01-07,14096.033,168.8,3.376857,59.470000,0.120782,0.120782,59.470000,3.376857,168.8,...,0.8660,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415,0.8445,0.8400,0.8545
4,2000-01-10,14096.033,168.8,3.376857,59.470000,0.120782,0.120782,59.470000,3.376857,168.8,...,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415,0.8445,0.8400,0.8545,0.8480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6032,2023-10-30,19374.000,307.0,6.600000,168.657489,0.136737,0.136737,168.657489,6.600000,307.0,...,3.6790,3.6380,3.6405,3.5870,3.6660,3.6835,3.7185,3.7025,3.7345,3.8335
6033,2023-10-31,19374.000,307.0,6.600000,168.657489,0.136676,0.136676,168.657489,6.600000,307.0,...,3.6380,3.6405,3.5870,3.6660,3.6835,3.7185,3.7025,3.7345,3.8335,3.8395
6034,2023-11-01,19374.000,307.0,6.600000,168.657489,0.136659,0.136659,168.657489,6.600000,307.0,...,3.6405,3.5870,3.6660,3.6835,3.7185,3.7025,3.7345,3.8335,3.8395,3.7970
6035,2023-11-02,19374.000,307.0,6.600000,168.657489,0.136687,0.136687,168.657489,6.600000,307.0,...,3.5870,3.6660,3.6835,3.7185,3.7025,3.7345,3.8335,3.8395,3.7970,3.7743


In [91]:
print(flipped_df)

       DEXCHUS  PALLFNFINDEXM  inflation_rate    CPI   Real_GDP  Price(t-15)  \
0     0.120774      59.470000        3.376857  168.8  14096.033       0.8480   
1     0.120776      59.470000        3.376857  168.8  14096.033       0.8565   
2     0.120777      59.470000        3.376857  168.8  14096.033       0.8530   
3     0.120782      59.470000        3.376857  168.8  14096.033       0.8540   
4     0.120782      59.470000        3.376857  168.8  14096.033       0.8465   
...        ...            ...             ...    ...        ...          ...   
6032  0.136737     168.657489        6.600000  307.0  19374.000       3.6585   
6033  0.136676     168.657489        6.600000  307.0  19374.000       3.6490   
6034  0.136659     168.657489        6.600000  307.0  19374.000       3.6490   
6035  0.136687     168.657489        6.600000  307.0  19374.000       3.6725   
6036  0.136977     168.657489        6.600000  307.0  19374.000       3.6815   

      Price(t-14)  Price(t-13)  Price(t

In [35]:
flipped_df_before_scaling_as_np = flipped_df.to_numpy()
flipped_df_before_scaling_as_np

array([[1.20774405e-01, 5.94700000e+01, 3.37685727e+00, ...,
        8.56500000e-01, 8.42500000e-01, 8.41500000e-01],
       [1.20775864e-01, 5.94700000e+01, 3.37685727e+00, ...,
        8.42500000e-01, 8.41500000e-01, 8.44500000e-01],
       [1.20777323e-01, 5.94700000e+01, 3.37685727e+00, ...,
        8.41500000e-01, 8.44500000e-01, 8.40000000e-01],
       ...,
       [1.36658695e-01, 1.68657489e+02, 6.60000000e+00, ...,
        3.83350000e+00, 3.83950000e+00, 3.79700000e+00],
       [1.36686714e-01, 1.68657489e+02, 6.60000000e+00, ...,
        3.83950000e+00, 3.79700000e+00, 3.77430000e+00],
       [1.36976919e-01, 1.68657489e+02, 6.60000000e+00, ...,
        3.79700000e+00, 3.77430000e+00, 3.82050000e+00]])

In [92]:
columns_to_extract = ['Price(t-15)', 'Price(t-14)', 'Price(t-13)', 'Price(t-12)', 'Price(t-11)',
                      'Price(t-10)', 'Price(t-9)', 'Price(t-8)', 'Price(t-7)', 'Price(t-6)',
                      'Price(t-5)', 'Price(t-4)', 'Price(t-3)', 'Price(t-2)', 'Price(t-1)']

# Create a new DataFrame with the desired columns
historic_prices = flipped_df[columns_to_extract].copy()
historic_prices

,Price(t-15),Price(t-14),Price(t-13),Price(t-12),Price(t-11),Price(t-10),Price(t-9),Price(t-8),Price(t-7),Price(t-6),Price(t-5),Price(t-4),Price(t-3),Price(t-2),Price(t-1)
0,0.8480,0.8565,0.8530,0.8540,0.8465,0.8435,0.8660,0.8565,0.8625,0.8660,0.8825,0.8730,0.8730,0.8565,0.8425
1,0.8565,0.8530,0.8540,0.8465,0.8435,0.8660,0.8565,0.8625,0.8660,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415
2,0.8530,0.8540,0.8465,0.8435,0.8660,0.8565,0.8625,0.8660,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415,0.8445
3,0.8540,0.8465,0.8435,0.8660,0.8565,0.8625,0.8660,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415,0.8445,0.8400
4,0.8465,0.8435,0.8660,0.8565,0.8625,0.8660,0.8825,0.8730,0.8730,0.8565,0.8425,0.8415,0.8445,0.8400,0.8545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6032,3.6585,3.6490,3.6490,3.6725,3.6815,3.7190,3.6790,3.6380,3.6405,3.5870,3.6660,3.6835,3.7185,3.7025,3.7345
6033,3.6490,3.6490,3.6725,3.6815,3.7190,3.6790,3.6380,3.6405,3.5870,3.6660,3.6835,3.7185,3.7025,3.7345,3.8335
6034,3.6490,3.6725,3.6815,3.7190,3.6790,3.6380,3.6405,3.5870,3.6660,3.6835,3.7185,3.7025,3.7345,3.8335,3.8395
6035,3.6725,3.6815,3.7190,3.6790,3.6380,3.6405,3.5870,3.6660,3.6835,3.7185,3.7025,3.7345,3.8335,3.8395,3.7970


In [93]:
historic_prices_as_np = historic_prices.to_numpy()
historic_prices_as_np

array([[0.848 , 0.8565, 0.853 , ..., 0.873 , 0.8565, 0.8425],
       [0.8565, 0.853 , 0.854 , ..., 0.8565, 0.8425, 0.8415],
       [0.853 , 0.854 , 0.8465, ..., 0.8425, 0.8415, 0.8445],
       ...,
       [3.649 , 3.6725, 3.6815, ..., 3.7345, 3.8335, 3.8395],
       [3.6725, 3.6815, 3.719 , ..., 3.8335, 3.8395, 3.797 ],
       [3.6815, 3.719 , 3.679 , ..., 3.8395, 3.797 , 3.7743]])

In [94]:
price_scaler_15 = MinMaxScaler(feature_range=(-1, 1))
scaled_prices_15 = price_scaler_15.fit_transform(historic_prices_as_np)

In [37]:
scaled_prices_15

array([[-0.89235389, -0.89189189, -0.88542389, ..., -0.88704089,
        -0.88542389, -0.88935089],
       [-0.89096789, -0.89235389, -0.89189189, ..., -0.88657889,
        -0.88704089, -0.88542389],
       [-0.89304689, -0.89096789, -0.89235389, ..., -0.89004389,
        -0.88657889, -0.88704089],
       ...,
       [ 0.47308847,  0.49272349,  0.48995149, ...,  0.41972742,
         0.41556942,  0.4047124 ],
       [ 0.46260106,  0.47308847,  0.49272349, ...,  0.43705244,
         0.41972742,  0.41556942],
       [ 0.48394548,  0.46260106,  0.47308847, ...,  0.41857242,
         0.43705244,  0.41972742]])

In [100]:
result = np.concatenate((scaled_GDP, scaled_CPI, scaled_IR, scaled_PALLFNFINDEXM,scaled_DEXCHUS, scaled_prices_flipped), axis=1)
result

array([[-1.        , -1.        , -0.10690381, ..., -0.88542389,
        -0.89189189, -0.89235389],
       [-1.        , -1.        , -0.10690381, ..., -0.89189189,
        -0.89235389, -0.89096789],
       [-1.        , -1.        , -0.10690381, ..., -0.89235389,
        -0.89096789, -0.89304689],
       ...,
       [ 0.36628638,  0.98864658,  0.66433555, ...,  0.48995149,
         0.49272349,  0.47308847],
       [ 0.36628638,  0.98864658,  0.66433555, ...,  0.49272349,
         0.47308847,  0.46260106],
       [ 0.36628638,  0.98864658,  0.66433555, ...,  0.47308847,
         0.46260106,  0.48394548]])

In [101]:
X = result[:, 0:LOOKBACK+4]
y = result[:, LOOKBACK+5]
X = X.astype(np.float32)
y = y.astype(np.float32)
X.shape

(6037, 19)

In [102]:
X

array([[-1.        , -1.        , -0.10690381, ..., -0.8778009 ,
        -0.8778009 , -0.8854239 ],
       [-1.        , -1.        , -0.10690381, ..., -0.8778009 ,
        -0.8854239 , -0.8918919 ],
       [-1.        , -1.        , -0.10690381, ..., -0.8854239 ,
        -0.8918919 , -0.8923539 ],
       ...,
       [ 0.36628637,  0.98864657,  0.66433555, ...,  0.42942944,
         0.44421345,  0.4899515 ],
       [ 0.36628637,  0.98864657,  0.66433555, ...,  0.44421345,
         0.4899515 ,  0.4927235 ],
       [ 0.36628637,  0.98864657,  0.66433555, ...,  0.4899515 ,
         0.4927235 ,  0.47308847]], dtype=float32)

In [103]:

split_index = int(len(X) * 0.80)
split_date = dates[split_index]

X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]
train_dates = dates[:split_index]
test_dates = dates[split_index:]
train_dates

0      2000-01-26
1      2000-01-27
2      2000-01-28
3      2000-01-31
4      2000-02-01
          ...    
4824   2019-03-29
4825   2019-04-01
4826   2019-04-02
4827   2019-04-03
4828   2019-04-04
Name: Date, Length: 4829, dtype: datetime64[ns]

In [104]:
X_train.shape

(4829, 19)

In [105]:
X = X.reshape((-1, LOOKBACK+4, 1))
X_train = X_train.reshape((-1, LOOKBACK+4, 1))
X_test = X_test.reshape((-1, LOOKBACK+4, 1))

y = y.reshape((-1, 1))
y_train = y_train.reshape((-1, 1))
y_test = y_test.reshape((-1, 1))
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4829, 19, 1), (1208, 19, 1), (4829, 1), (1208, 1))

In [106]:
X = torch.tensor(X).float()
y = torch.tensor(y).float()
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()

X_test.dtype

torch.float32

In [107]:
y_train.shape

torch.Size([4829, 1])

In [108]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

train_dataset = TimeSeriesDataset(X, y)
test_dataset = TimeSeriesDataset(X_test, y_test)

In [109]:
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [110]:
for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.shape, y_batch.shape)
    break

torch.Size([8, 19, 1]) torch.Size([8, 1])


In [111]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers,
                            batch_first=True)

        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model = LSTM(1, 12, 1)
model.to(device)
model

LSTM(
  (lstm): LSTM(1, 12, batch_first=True)
  (fc): Linear(in_features=12, out_features=1, bias=True)
)

In [112]:
def train_one_epoch():
    model.train(True)
    print(f'Epoch: {epoch + 1}')
    running_loss = 0.0

    for batch_index, batch in enumerate(train_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)
        output = model(x_batch)
        loss = loss_function(output, y_batch)
        running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_index % 100 == 99:  # print every 100 batches
            avg_loss_across_batches = running_loss / 100
            print('Batch {0}, Loss: {1:.5f}'.format(batch_index+1,
                                                    avg_loss_across_batches))
            running_loss = 0.0
    print()

In [113]:
def validate_one_epoch():
    model.train(False)
    running_loss = 0.0

    for batch_index, batch in enumerate(test_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        with torch.no_grad():
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(test_loader)

    print('Val Loss: {0:.5f}'.format(avg_loss_across_batches))
    print('***************************************************')
    print()

In [114]:
learning_rate = 0.001
num_epochs = 30
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#Adam does the gradient descent

for epoch in range(num_epochs):
    train_one_epoch()
    validate_one_epoch()

Epoch: 1
Batch 100, Loss: 0.19229
Batch 200, Loss: 0.01532
Batch 300, Loss: 0.00570
Batch 400, Loss: 0.00251
Batch 500, Loss: 0.00206
Batch 600, Loss: 0.00217
Batch 700, Loss: 0.00173

Val Loss: 0.00238
***************************************************

Epoch: 2
Batch 100, Loss: 0.00161
Batch 200, Loss: 0.00176
Batch 300, Loss: 0.00152
Batch 400, Loss: 0.00175
Batch 500, Loss: 0.00144
Batch 600, Loss: 0.00145
Batch 700, Loss: 0.00147

Val Loss: 0.00200
***************************************************

Epoch: 3
Batch 100, Loss: 0.00129
Batch 200, Loss: 0.00138
Batch 300, Loss: 0.00163
Batch 400, Loss: 0.00154
Batch 500, Loss: 0.00155
Batch 600, Loss: 0.00121
Batch 700, Loss: 0.00131

Val Loss: 0.00178
***************************************************

Epoch: 4
Batch 100, Loss: 0.00124
Batch 200, Loss: 0.00121
Batch 300, Loss: 0.00145
Batch 400, Loss: 0.00115
Batch 500, Loss: 0.00129
Batch 600, Loss: 0.00134
Batch 700, Loss: 0.00139

Val Loss: 0.00152
*****************************

In [115]:
with torch.no_grad():
    predicted = model(X[:].to(device)).to('cpu').numpy()




In [116]:
train_predictions = predicted.flatten()

dummies = np.zeros((X.shape[0], LOOKBACK+1))
dummies[:, 0] = train_predictions 
print(dummies[0])
dummies = prices_scaler.inverse_transform(dummies)

train_predictions = dc(dummies[:, 0])
train_predictions

[-0.88901949  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.        ]


array([0.84871732, 0.83604799, 0.83259402, ..., 3.83571163, 3.84688646,
       3.80678638])

In [117]:
dummies = np.zeros((X.shape[0], LOOKBACK+1))
dummies[:, 0] = y.flatten()
dummies = prices_scaler.inverse_transform(dummies)
new_y_train = dc(dummies[:, 0])
new_y_train

array([0.8415    , 0.84449997, 0.83999995, ..., 3.797     , 3.77430001,
       3.82050001])

In [118]:
plt.plot(dates,y,  label='Actual Price')
plt.plot(dates,train_predictions, label='Predicted Price')
plt.xlabel('Day')
plt.ylabel('Price')
plt.legend()
plt.show()

In [119]:
test_predictions = model(X.to(device)).detach().cpu().numpy().flatten()

dummies = np.zeros((X.shape[0], LOOKBACK+1))
dummies[:, 0] = test_predictions
dummies = prices_scaler.inverse_transform(dummies)

test_predictions = dc(dummies[:, 0])
test_predictions

array([0.84871732, 0.83604799, 0.83259402, ..., 3.83571163, 3.84688646,
       3.80678638])

In [120]:
dummies = np.zeros((X.shape[0], LOOKBACK+1))
dummies[:, 0] = y.flatten()
dummies = prices_scaler.inverse_transform(dummies)

new_y_test = dc(dummies[:, 0])
new_y_test
squared_differences = (new_y_test - test_predictions)**2

# Calculate the Root Mean Squared Error (RMSE)
rmse = np.sqrt(np.mean(squared_differences))
#rmse
absolute_differences = np.abs(new_y_test - test_predictions)

# Calculate the Mean Absolute Error (MAE)
mae = np.mean(absolute_differences)
rmse

0.06513532158285033

In [121]:
plt.figure(figsize=(10,6))
plt.plot(dates[split_index:], new_y_test[split_index:], label='Actual Price')
plt.plot(dates[split_index:],test_predictions[split_index:], label='Predicted Price')
plt.xlabel('Day')
plt.ylabel('Price')
plt.legend()
plt.show()

In [122]:
torch.save(model, 'model.pt')

In [123]:

m1 = torch.load('model.pt', map_location=torch.device('cpu'))

In [124]:
model

LSTM(
  (lstm): LSTM(1, 12, batch_first=True)
  (fc): Linear(in_features=12, out_features=1, bias=True)
)

In [125]:
model=m1

In [126]:
start_date = last_date+timedelta(days=1)
end_date = "2024-03-31"
forecast_dates = pd.date_range(start=start_date, end=end_date, freq='B')  # 'B' excludes weekends



# Create a DataFrame with the date column
forecast_set = pd.DataFrame({'Date': forecast_dates})
forecast_set['Date'] = pd.to_datetime(forecast_set['Date'])
forecast_set

,Date
0,2023-11-27
1,2023-11-28
2,2023-11-29
3,2023-11-30
4,2023-12-01
...,...
85,2024-03-25
86,2024-03-26
87,2024-03-27
88,2024-03-28


In [127]:
forecast_dates.shape

(90,)

In [128]:
stacked_np.dtype

dtype('float64')

In [129]:
np.save('stacked_np.npy', stacked_np)

In [130]:
shifted_df_as_np_flipped = np.fliplr(stacked_np)
shifted_df_as_np_flipped

array([[-0.99993486, -0.88354852, -0.10690381, ..., -0.88542389,
        -0.89189189, -0.89235389],
       [-0.99986972, -0.88354852, -0.10690381, ..., -0.89189189,
        -0.89235389, -0.89096789],
       [-0.99980458, -0.88354852, -0.10690381, ..., -0.89235389,
        -0.89096789, -0.89304689],
       ...,
       [-0.29056913,  0.24367179,  0.66433555, ...,  0.48995149,
         0.49272349,  0.47308847],
       [-0.28931784,  0.24367179,  0.66433555, ...,  0.49272349,
         0.47308847,  0.46260106],
       [-0.27635776,  0.24367179,  0.66433555, ...,  0.47308847,
         0.46260106,  0.48394548]])

In [131]:
shifted_df_as_tensor = torch.tensor(shifted_df_as_np_flipped.copy())
torch.set_printoptions(precision=6)
last = shifted_df_as_tensor[-1][1:]
last = last.unsqueeze(0).unsqueeze(-1)
last.shape

torch.Size([1, 20, 1])

In [132]:
shifted_df_as_tensor.shape

torch.Size([6037, 21])

In [133]:
shifted_df

,Date,Price,Price(t-1),Price(t-2),Price(t-3),Price(t-4),Price(t-5),Price(t-6),Price(t-7),Price(t-8),...,Price(t-11),Price(t-12),Price(t-13),Price(t-14),Price(t-15),Real_GDP,CPI,inflation_rate,PALLFNFINDEXM,DEXCHUS
0,2000-01-26,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,0.8565,...,0.8465,0.8540,0.8530,0.8565,0.8480,14096.033,168.8,3.376857,59.470000,0.120774
1,2000-01-27,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,...,0.8435,0.8465,0.8540,0.8530,0.8565,14096.033,168.8,3.376857,59.470000,0.120776
2,2000-01-28,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,...,0.8660,0.8435,0.8465,0.8540,0.8530,14096.033,168.8,3.376857,59.470000,0.120777
3,2000-01-31,0.8545,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,...,0.8565,0.8660,0.8435,0.8465,0.8540,14096.033,168.8,3.376857,59.470000,0.120782
4,2000-02-01,0.8480,0.8545,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,...,0.8625,0.8565,0.8660,0.8435,0.8465,14096.033,168.8,3.376857,59.470000,0.120782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6032,2023-11-20,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,3.6380,...,3.6815,3.6725,3.6490,3.6490,3.6585,19374.000,307.0,6.600000,168.657489,0.136737
6033,2023-11-21,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,...,3.7190,3.6815,3.6725,3.6490,3.6490,19374.000,307.0,6.600000,168.657489,0.136676
6034,2023-11-22,3.7970,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,...,3.6790,3.7190,3.6815,3.6725,3.6490,19374.000,307.0,6.600000,168.657489,0.136659
6035,2023-11-23,3.7743,3.7970,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,...,3.6380,3.6790,3.7190,3.6815,3.6725,19374.000,307.0,6.600000,168.657489,0.136687


In [134]:
shifted_df.to_csv('shifted_df.csv', index=False, float_format='%.5f')

In [135]:
shifted_df_no_date = shifted_df.drop(['Date'], axis=1)
shifted_df_no_date

,Price,Price(t-1),Price(t-2),Price(t-3),Price(t-4),Price(t-5),Price(t-6),Price(t-7),Price(t-8),Price(t-9),...,Price(t-11),Price(t-12),Price(t-13),Price(t-14),Price(t-15),Real_GDP,CPI,inflation_rate,PALLFNFINDEXM,DEXCHUS
0,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,0.8565,0.8660,...,0.8465,0.8540,0.8530,0.8565,0.8480,14096.033,168.8,3.376857,59.470000,0.120774
1,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,0.8565,...,0.8435,0.8465,0.8540,0.8530,0.8565,14096.033,168.8,3.376857,59.470000,0.120776
2,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,0.8625,...,0.8660,0.8435,0.8465,0.8540,0.8530,14096.033,168.8,3.376857,59.470000,0.120777
3,0.8545,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,0.8660,...,0.8565,0.8660,0.8435,0.8465,0.8540,14096.033,168.8,3.376857,59.470000,0.120782
4,0.8480,0.8545,0.8400,0.8445,0.8415,0.8425,0.8565,0.8730,0.8730,0.8825,...,0.8625,0.8565,0.8660,0.8435,0.8465,14096.033,168.8,3.376857,59.470000,0.120782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6032,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,3.6380,3.6790,...,3.6815,3.6725,3.6490,3.6490,3.6585,19374.000,307.0,6.600000,168.657489,0.136737
6033,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,3.6380,...,3.7190,3.6815,3.6725,3.6490,3.6490,19374.000,307.0,6.600000,168.657489,0.136676
6034,3.7970,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,3.6405,...,3.6790,3.7190,3.6815,3.6725,3.6490,19374.000,307.0,6.600000,168.657489,0.136659
6035,3.7743,3.7970,3.8395,3.8335,3.7345,3.7025,3.7185,3.6835,3.6660,3.5870,...,3.6380,3.6790,3.7190,3.6815,3.6725,19374.000,307.0,6.600000,168.657489,0.136687


In [136]:
import joblib
joblib.dump(prices_scaler, 'prices_scaler.pkl')

['prices_scaler.pkl']

## Forecasting

In [137]:
for i in range(forecast_dates.shape[0]):
    last = shifted_df_as_tensor[-1][1:]
    last = last.unsqueeze(0).unsqueeze(-1).float()
    with torch.no_grad():
        predicted = model(last.to(device)).to('cpu').numpy()
    row = torch.cat([shifted_df_as_tensor[-1][1:],torch.tensor(predicted).reshape(1)],dim=0).unsqueeze(0)
    quick_reverse = prices_scaler.inverse_transform(row[:,5:].to('cpu').numpy())
    new_row = {'Price':quick_reverse[0][LOOKBACK], 
               'Price(t-1)':quick_reverse[0][LOOKBACK-1],
               'Price(t-2)':quick_reverse[0][LOOKBACK-2],
               'Price(t-3)':quick_reverse[0][LOOKBACK-3],
               'Price(t-4)':quick_reverse[0][LOOKBACK-4],
               'Price(t-5)':quick_reverse[0][LOOKBACK-5],
               'Price(t-6)':quick_reverse[0][LOOKBACK-6],
               'Price(t-7)':quick_reverse[0][LOOKBACK-7],
               'Price(t-8)':quick_reverse[0][LOOKBACK-8],
               'Price(t-9)':quick_reverse[0][LOOKBACK-9],
               'Price(t-10)':quick_reverse[0][LOOKBACK-10],
               'Price(t-11)':quick_reverse[0][LOOKBACK-11],
               'Price(t-12)':quick_reverse[0][LOOKBACK-12],
               'Price(t-13)':quick_reverse[0][LOOKBACK-13],
               'Price(t-14)':quick_reverse[0][LOOKBACK-14],
               'Price(t-15)':quick_reverse[0][LOOKBACK-15],
               'Real_GDP':df.iloc[-1]["Real_GDP"],
               'CPI':df.iloc[-1]["CPI"],
               'inflation_rate':df.iloc[-1]["inflation_rate"],
               'PALLFNFINDEXM':df.iloc[-1]["PALLFNFINDEXM"],
               'DEXCHUS':df.iloc[-1]["DEXCHUS"]
               }
    shifted_df_as_tensor = torch.cat((shifted_df_as_tensor, row),dim=0)
    shifted_df_no_date = shifted_df_no_date._append(new_row,ignore_index=True)
shifted_df_no_date
    

,Price,Price(t-1),Price(t-2),Price(t-3),Price(t-4),Price(t-5),Price(t-6),Price(t-7),Price(t-8),Price(t-9),...,Price(t-11),Price(t-12),Price(t-13),Price(t-14),Price(t-15),Real_GDP,CPI,inflation_rate,PALLFNFINDEXM,DEXCHUS
0,0.841500,0.842500,0.856500,0.873000,0.873000,0.882500,0.866000,0.862500,0.856500,0.866000,...,0.846500,0.854000,0.853000,0.856500,0.848000,14096.033,168.8,3.376857,59.470000,0.120774
1,0.844500,0.841500,0.842500,0.856500,0.873000,0.873000,0.882500,0.866000,0.862500,0.856500,...,0.843500,0.846500,0.854000,0.853000,0.856500,14096.033,168.8,3.376857,59.470000,0.120776
2,0.840000,0.844500,0.841500,0.842500,0.856500,0.873000,0.873000,0.882500,0.866000,0.862500,...,0.866000,0.843500,0.846500,0.854000,0.853000,14096.033,168.8,3.376857,59.470000,0.120777
3,0.854500,0.840000,0.844500,0.841500,0.842500,0.856500,0.873000,0.873000,0.882500,0.866000,...,0.856500,0.866000,0.843500,0.846500,0.854000,14096.033,168.8,3.376857,59.470000,0.120782
4,0.848000,0.854500,0.840000,0.844500,0.841500,0.842500,0.856500,0.873000,0.873000,0.882500,...,0.862500,0.856500,0.866000,0.843500,0.846500,14096.033,168.8,3.376857,59.470000,0.120782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6122,4.363678,4.360771,4.357788,4.354729,4.351592,4.348376,4.345079,4.341701,4.338241,4.334697,...,4.327352,4.323550,4.319659,4.315680,4.311610,19374.000,307.0,6.600000,168.657489,0.136977
6123,4.366511,4.363678,4.360771,4.357788,4.354729,4.351592,4.348376,4.345079,4.341701,4.338241,...,4.331067,4.327352,4.323550,4.319659,4.315680,19374.000,307.0,6.600000,168.657489,0.136977
6124,4.369272,4.366511,4.363678,4.360771,4.357788,4.354729,4.351592,4.348376,4.345079,4.341701,...,4.334697,4.331067,4.327352,4.323550,4.319659,19374.000,307.0,6.600000,168.657489,0.136977
6125,4.371962,4.369272,4.366511,4.363678,4.360771,4.357788,4.354729,4.351592,4.348376,4.345079,...,4.338241,4.334697,4.331067,4.327352,4.323550,19374.000,307.0,6.600000,168.657489,0.136977


In [138]:
df

,Date,Month,Year,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM,DEXCHUS
0,2000-01-04,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120774
1,2000-01-05,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120776
2,2000-01-06,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120777
3,2000-01-07,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120782
4,2000-01-10,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120782
...,...,...,...,...,...,...,...,...
6032,2023-10-30,10.0,2023.0,19374.000,307.0,6.600000,168.657489,0.136737
6033,2023-10-31,10.0,2023.0,19374.000,307.0,6.600000,168.657489,0.136676
6034,2023-11-01,11.0,2023.0,19374.000,307.0,6.600000,168.657489,0.136659
6035,2023-11-02,11.0,2023.0,19374.000,307.0,6.600000,168.657489,0.136687


In [139]:
shifted_df_as_tensor.shape

torch.Size([6127, 21])

## Inverse Scaling

In [140]:
final_set = prices_scaler.inverse_transform(shifted_df_as_tensor[:,5:].to('cpu').numpy())
final_set

array([[0.848     , 0.8565    , 0.853     , ..., 0.8565    , 0.8425    ,
        0.8415    ],
       [0.8565    , 0.853     , 0.854     , ..., 0.8425    , 0.8415    ,
        0.8445    ],
       [0.853     , 0.854     , 0.8465    , ..., 0.8415    , 0.8445    ,
        0.84      ],
       ...,
       [4.31965913, 4.32354968, 4.32735225, ..., 4.36367801, 4.36651143,
        4.36927233],
       [4.32354968, 4.32735225, 4.33106747, ..., 4.36651143, 4.36927233,
        4.37196176],
       [4.32735225, 4.33106747, 4.33469677, ..., 4.36927233, 4.37196176,
        4.37458127]])

In [141]:
shifted_df_no_date

,Price,Price(t-1),Price(t-2),Price(t-3),Price(t-4),Price(t-5),Price(t-6),Price(t-7),Price(t-8),Price(t-9),...,Price(t-11),Price(t-12),Price(t-13),Price(t-14),Price(t-15),Real_GDP,CPI,inflation_rate,PALLFNFINDEXM,DEXCHUS
0,0.841500,0.842500,0.856500,0.873000,0.873000,0.882500,0.866000,0.862500,0.856500,0.866000,...,0.846500,0.854000,0.853000,0.856500,0.848000,14096.033,168.8,3.376857,59.470000,0.120774
1,0.844500,0.841500,0.842500,0.856500,0.873000,0.873000,0.882500,0.866000,0.862500,0.856500,...,0.843500,0.846500,0.854000,0.853000,0.856500,14096.033,168.8,3.376857,59.470000,0.120776
2,0.840000,0.844500,0.841500,0.842500,0.856500,0.873000,0.873000,0.882500,0.866000,0.862500,...,0.866000,0.843500,0.846500,0.854000,0.853000,14096.033,168.8,3.376857,59.470000,0.120777
3,0.854500,0.840000,0.844500,0.841500,0.842500,0.856500,0.873000,0.873000,0.882500,0.866000,...,0.856500,0.866000,0.843500,0.846500,0.854000,14096.033,168.8,3.376857,59.470000,0.120782
4,0.848000,0.854500,0.840000,0.844500,0.841500,0.842500,0.856500,0.873000,0.873000,0.882500,...,0.862500,0.856500,0.866000,0.843500,0.846500,14096.033,168.8,3.376857,59.470000,0.120782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6122,4.363678,4.360771,4.357788,4.354729,4.351592,4.348376,4.345079,4.341701,4.338241,4.334697,...,4.327352,4.323550,4.319659,4.315680,4.311610,19374.000,307.0,6.600000,168.657489,0.136977
6123,4.366511,4.363678,4.360771,4.357788,4.354729,4.351592,4.348376,4.345079,4.341701,4.338241,...,4.331067,4.327352,4.323550,4.319659,4.315680,19374.000,307.0,6.600000,168.657489,0.136977
6124,4.369272,4.366511,4.363678,4.360771,4.357788,4.354729,4.351592,4.348376,4.345079,4.341701,...,4.334697,4.331067,4.327352,4.323550,4.319659,19374.000,307.0,6.600000,168.657489,0.136977
6125,4.371962,4.369272,4.366511,4.363678,4.360771,4.357788,4.354729,4.351592,4.348376,4.345079,...,4.338241,4.334697,4.331067,4.327352,4.323550,19374.000,307.0,6.600000,168.657489,0.136977


In [142]:
display_df = pd.DataFrame(shifted_df_no_date["Price"])
display_df

,Price
0,0.841500
1,0.844500
2,0.840000
3,0.854500
4,0.848000
...,...
6122,4.363678
6123,4.366511
6124,4.369272
6125,4.371962


In [143]:
display_df = display_df.iloc[TRAIN_DATA_LEN:]
display_df

,Price
6037,3.827056
6038,3.836283
6039,3.844766
6040,3.853090
6041,3.861417
...,...
6122,4.363678
6123,4.366511
6124,4.369272
6125,4.371962


In [144]:

display_df = display_df.reset_index(drop=True)

In [145]:
display_df

,Price
0,3.827056
1,3.836283
2,3.844766
3,3.853090
4,3.861417
...,...
85,4.363678
86,4.366511
87,4.369272
88,4.371962


In [146]:
forecast_set = pd.concat([forecast_set, display_df],axis=1)

In [147]:
forecast_set

,Date,Price
0,2023-11-27,3.827056
1,2023-11-28,3.836283
2,2023-11-29,3.844766
3,2023-11-30,3.853090
4,2023-12-01,3.861417
...,...,...
85,2024-03-25,4.363678
86,2024-03-26,4.366511
87,2024-03-27,4.369272
88,2024-03-28,4.371962


In [148]:
df

,Date,Month,Year,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM,DEXCHUS
0,2000-01-04,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120774
1,2000-01-05,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120776
2,2000-01-06,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120777
3,2000-01-07,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120782
4,2000-01-10,1.0,2000.0,14096.033,168.8,3.376857,59.470000,0.120782
...,...,...,...,...,...,...,...,...
6032,2023-10-30,10.0,2023.0,19374.000,307.0,6.600000,168.657489,0.136737
6033,2023-10-31,10.0,2023.0,19374.000,307.0,6.600000,168.657489,0.136676
6034,2023-11-01,11.0,2023.0,19374.000,307.0,6.600000,168.657489,0.136659
6035,2023-11-02,11.0,2023.0,19374.000,307.0,6.600000,168.657489,0.136687
